## Current VAEs and Objective

**Current VAEs methods can be divided into two categories depending on how they deal will correlated latents**:

1.**(Full)** Methods that don't care about correlations at all and make no attempt at resolving them. Vanilla VAE with single latent layer and Hierarchichal VAEs like LadderVAE, BIVA etc. are examples of this class. Usually use The use fully conditioned priors.

2.**(None)** Methods that deal with mild or synthetically constructed correlations and attempt to resolve all the correlations entirely, mostly because of fairness reasons. and do it post-hoc. Ada-Beta-VAE is an example of this class which relies on Beta-VAE based methods. The use independent priors.

**Some observations**:

- In real world, latent structure is not fully connected. Latent interactions are sparse and more meaningful
- Even with fairness in mind, not all correlations are harmful. We might want to resolve *some* correlations (e.g. for senstitive attributes) and *preserve* some other correlations (imposed by nature, environment etc.)

**We care about two things:**
1. Sparse and meaningful/interpretable latent interactions
2. Possibility to remove unwanted correlations



## An Example to Illustrate

**As an Example, assume the following true latent structure**:
<img src="norm_dag.jpg" alt="Latent Structure" width="300" height="300"/>

Further assume that we regard (gender, height) pair as sensitive and would want to learn a latent structure which doesn't have this association (as shown below):
<img src="marg_dag.jpg" alt="Latent Structure" width="300" height="300"/>

**Such multi-layered latent structure can be specified by hierarchical VAEs e.g. Ladder VAE**

**But they have a serious limitation w.r.t. our objective**. The latent hierarchy is fully-conditional i.e every dimension of layer 1 is dependent on all the dimensions in layer 2. That is, the interactions are **not sparse and not meaningful**.

In math:

\begin{align*}
p(\textbf{z}_1 | \textbf{z}_2) &= p( z_1^1, z_1^2, z_1^3 |  z_2^1, z_2^2, z_2^3) \\
&= p(z_1 | z_2^1, z_2^2, z_2^3) \times p(z_1 | z_2^1, z_2^2, z_2^3) \times p(z_1 | z_2^1, z_2^2, z_2^3)
\end{align*}


As seen in the example, we want more structured dependencies than this. An example of what we want would be this:

Let,

  \begin{align}
    \textbf{z}_2 = \begin{bmatrix}
           age \\
           gender
         \end{bmatrix} \quad
    \textbf{z}_1 = \begin{bmatrix}
           edu level \\
           height \\
           hair length
         \end{bmatrix} 
\end{align}

\begin{align*}
p(\textbf{z}_1 | \textbf{z}_2) &= p(\texttt{age}) \times  p(\texttt{gender}) \\ &\times p(\texttt{edu level} | \texttt{age}) \times p(\texttt{hair len} | \texttt{gender}) \times p(\texttt{height} | \texttt{age, gender})
\end{align*}


## How to impose such structure?

For this we need :
1. **A VAE which allows layered latents** 
2. **Structured Prior** and  
3. **Structured Variational Distributions**

**Ans 1 - We can adapt Ladder VAE to represent layered latents**

**Ans 2 and 3 - How to find the prior structure and How to impose that structure in the variational distributions?** 


## Finding Structure

For this we have two approaches 
- (1) Max likelihood Bayesian Structure estimation via Chow-Liu algo
- (2) K Active Parents via $l_0$ regularization


### (1) Max likelihood Bayesian Structure estimation via Chow-Liu algo

**Given** : Latent Labels

**Assumption** : Latent structure is a tree (in case of trees, globally optimal (i.e. MLE) graph structure can be found via exact methods)

It can be extended to > 2 layers as long as the latent structure remains a tree. See the following:

<img src="tree.jpg" alt="Latent Structure" width="300" height="300"/>


**Goal**: Find latent structure and impose it on prior and variational dists...

Boils down to (Ch26, Kevin Murphy):
- Maximising the following objective (for bernoulli and categorial we can get distributions simply by counting)

$$ p(\textbf{z}) = \prod_v p(z_v) \times \prod_{(s,t)} \frac{p(z_s,z_t)}{p(z_s)\times p(z_t)} $$

- Compute maximum weight spanning tree where edge weights are pairwise MI computed from empirical distribution.



- **This pre-processing steps gives us a graph topology that can be represented as an adjacency matrix**. The topology and the matrix is symmetric. If we want to use this topology in a prior we need to convert it into a DAG adj. mat.

- If the number of latent variables in low we can do so by inspection and assign each latent to higher or lower layer as we like.

- If the latents are too many, we can use simple heuristics to create two groups. One example is to assign a variable to higher layer if it has a higher than avg degree as it means it is involved in many latent variables and likely to be independent itself.

- Once we have established a topology either by inspection or heuristics we can construct prior and posterior distributions which respect that topology.


As example assume that we have grouped the variables as such:
  \begin{align}
    \textbf{z}_1 = \begin{bmatrix}
           age \\
           gender
         \end{bmatrix} \quad
    \textbf{z}_2 = \begin{bmatrix}
           edu level \\
           height \\
           hair length
         \end{bmatrix} 
\end{align}
  
And the structure found is this (without the X):

<img src="norm_dag.jpg" alt="Latent Structure" width="300" height="300"/>


We modify ladder-vae as follows:

The top layer is represented as isotropic Gaussians.

**The parameters for lower layer are produced by an NN which respects the learned topology.**

$p(\textbf{z}_1 | \textbf{z}_2)$ and $q(\textbf{z}_1 | \textbf{z}_2, \textbf{x})$ is modelled by such NNs.



<img src="nnex.jpg" alt="Latent Structure" width="300" height="300"/>

Or more generally:
<img src="specialnet.jpg" alt="Latent Structure" width="300" height="300"/>


### (2) K active parents

- This approach doesn't require any latent labels. 
- This approach doesn't assume latents to have a tree structure. 
- This approach doesn't allow us to delete unwanted correlations

Idea behind K-active-parents is that each output unit is only allowed to depend on a maximum of k input units. 

<img src="kactive.jpg" alt="Latent Structure" width="600" height="300"/>

**Note that if we remove the violet edge, we no longer have a tree (rather a forest)**


If we have A input units and B output units with an intermediate hidden layer we can use the NN structure we saw in previous approach:

<img src="specialnet.jpg" alt="Latent Structure" width="300" height="300"/>

If the number of units in the box is $r$, then each box can have a maximum of $r \times k$ incoming non-zero weights.

**Regularizer**: We can penalize this using $l_0$ norm but it is not SGD-optimizable since it is not continuous. Thus to optimize this we need to use the surrogate $l_0$ loss introduced in Learning Sparse NN paper (Kigma and Welling, ICLR 2018).

Key idea: Multiply the weights with bernoulli random variables then minimize the "on" probability.